In [ ]:
%matplotlib inline


# Statistical Analyses

Apply statistical analyses to spiking data.

This tutorial primarily covers the ``spiketools.stats`` module.


## Applying statistical measures to spiking data

Words, words, words.




In [ ]:
import spiketools

In [ ]:
# Settings
...

### Subtitle

Words, words, words.




In [ ]:
# Do some code stuff
print('Hello World!')

In [ ]:
# Do more code stuff
print('More stuff!')

### Conclusion

Words, words, words.


